# Comunicacion de datos

**NOTA: El ambiente de Jupyter Notebook no es el espacio ideal de programación donde codificar programas de comunicación. Esta guía es más un documento de referencia con los códigos a ser probados en un ambiente como Spyder o la consola de comandos.**

Se puede establecer comunicación de datos entre perifericos utilizando diferentes medios y tecnologías. Python tiene diferentes librerias para poder intercambiar datos con otros elementos, utilizando dos tecnologías de transmisión de datos:

* Serial (COM, UART, USB)
* Sockets (Ethernet, WiFi)

## Serial
La coumunicación serial de datos (COM) utiliza un modo de transmisión bit a bit de forma asíncrona, full-duplex. Es muy utilizada en entornos industriales (bajo estandar RS-232 o RS-485) y en camaras y dispositivos externos de aplicaciones especiales. También como puertos de servicio y monitoreo en equipos de comunicaciones.

![](https://www.technologyuk.net/telecommunications/communication-technologies/images/rs-232_03.gif)

Los equipos que se conectan por medio de una interface serial requieren puertos fisicos que son conectados por medio de un cable y equipos de interconexión. Originalmente, utilizaban la red telefonica para establecer comunicación entre nodos remotos, por lo que la comunicación requerian de MODEMs (MOdulator/DEModulator) en ambos extremos. Si se unes dos puertos fisicos por un cable, suprimiendo los MODEMs, se tendrá un NULL MODEM.

![](https://panamahitek.com/wp-content/uploads/2016/01/device5.gif)

Una forma de poder simular una comunicación entre equipos utilizando interfases seriales es estableciendo una infraestructura virtual, en donde se tendrán dos puertos seriales conectados por un cable NULL MODEM a los que se podrá acceder para intercambiar información. Se puede utilizar un software para habilitar puertos seriales virtuales. En Windows, se puede utilizar la aplicacion [Free Virtual Serial Ports](https://freevirtualserialports.com/) para simular la infraestructura.

![](https://freevirtualserialports.com/images/products/screenshots/index-free-virtual-serial-ports.png)

En Python se necesita instalar una libreria que pueda gestionar el protocolo serial: `PySerial`. La [documentación de PySerial](https://pythonhosted.org/pyserial/) es breve e indica que un objeto Serial tiene los siguientes metodos básicos:

* open()
* close()
* readline()
* read(buffer)
* write(data)

Adicionalmente tiene los getters:

* in_waiting
* is_open

La libreria `PySerial` incluye un monitor serial para poder realizar pruebas, disponible desde terminal con la instrucción `python -m serial.tools.miniterm <port name>`, por ejemplo:

    python -m serial.tools.miniterm COM2

## Instalacion (pip / conda)
    python -m pip install pyserial
    conda install -c conda-forge pyserial
    
    The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    pyserial-3.4               |             py_2          61 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          61 KB


In [ ]:
# Las siguientes pruebas se basan en la habilitación de una infraestructura serial virtual
# utilizando un puerto serial espcificado en PORT. Debe de modificar esta variable para
# que se ajuste a su ambiente virtual (Crear un Bridge COM - COM)
import serial.tools.list_ports

ports = serial.tools.list_ports.comports()

for port in ports:
    print(port)

## Recepcion de datos (El script recibe datos de un dispositivo conectado al COM)

In [ ]:
# Asi tambien, debe de instalar pip install PySerial
import serial

PORT = "COM1"

try:
    print(f"Estableciendo comunicación con puerto serial {PORT}...")
    ser = serial.Serial(port=PORT, 
                        baudrate=9600, 
                        bytesize=8, 
                        timeout=2, 
                        stopbits=serial.STOPBITS_ONE)
    print("Conexión extablecida")

    while True:
        try:
            # Si hay datos en el butffer de entrada... 
            if ser.in_waiting > 0:
                # Se leen los datos y se espera el caracter EOL
                data = ser.readline()
                # Los datos recibidos son bytes. Para verlos es necesarios decodificarlos
                string = data.decode('utf-8')
                print(f"Rx: {string}")
        # Se calcela el programa con CTRL-C desde el terminal
        except KeyboardInterrupt:
            print("Conexion cerrada")
            ser.close()
            break
except:
    print(f"Puerto {PORT} no disponible")


In [ ]:
# Cierra el puerto en caso se quede abierto en las pruebas
ser.close()

## Envío de datos (El script envia datos al dispositivo conectado al COM)

In [ ]:
# La siguiente prueba se basa en la habilitación de una infraestructura serial virtual
# utilizando un puerto serial espcificado en PORT. Debe de modificar esta variable para
# que se ajuste a su ambiente virtual (Crear un Bridge COM - COM)

# Asi tambien, debe de instalar pip install PySerial
import serial

PORT = "COM1"

try:
    print(f"Estableciendo comunicación con puerto serial {PORT}...")
    ser = serial.Serial(port=PORT, 
                        baudrate=9600, 
                        bytesize=8, 
                        timeout=2, 
                        stopbits=serial.STOPBITS_ONE)
    print("Conexión extablecida")

    while True:
        # Se ingresa el texto a enviar por el puerto
        string = input("Datos a enviar:")
        # Se define una palabra para cerrar el puerto
        if string == "END":
            break
        else:
            # Se codifica como bytes el texto a enviar y se envia por el puerto
            data = string.encode("utf-8")
            ser.write(data)
    
    print("Conexion cerrada")
    ser.close()
except:
    print(f"Puerto {PORT} no disponible")


In [ ]:
# Cierra el puerto en caso se quede abierto en las pruebas
ser.close()

## Projecto Final - Parte 1
Escriba un programa en customtkinter de Chat Serial. Este debe de permitir establecer comunicación entre dos equipos utilizando un NULL MODEM, o con dos aplicaciones ejeuctadas de forma simultanea y conectadas a un Serial Bridge habilitado con emulador de puerto serial Virtual Serial Ports.

### CONDICIONES DE OPERACION:
* Al escribir o recibir se debe de mostar justificados a la izquierda con el formato ajustado al ancho de la ventana:
    
        {PORT}: {MENSAJE}
      
* Adicionalmente, los mensaje enviados y recibidos deben de tener colores diferentes.
   
* Si se realiza la conexion debe de bloquearse el OptionsMenu y el boton de "Conectar" debe pasar a mostrar "Desconectar" y en caso se le haga click liberará la conexión y los controles retornaran a sus estado normal.
* Si la App esta conectada al puerto serial, los controles del frm3 deben de habilitarse. De lo contrario, estarán deshabilitados.
* El OptionsMenu debe de mostrar los puertos COM instalados en el sistema

Se muestra un prototipo de aplicación que deberá completar con todas las funcionalidades requeridas.

In [6]:
from datetime import datetime
import customtkinter as ctk
import serial

class SerialChat(ctk.CTk):
    def __init__(self):
        super().__init__()
        self.PORT = "COM1"
        
        self.title(f"Serial Chat App")
        self.geometry("+50+50")
        self.resizable(0, 0)
        
        ctk.set_appearance_mode("dark")
        # ---------------------- SERIAL PORT --------------------------
        self.serial = None
        
        # ------------------------ FRAMES -----------------------------
        frm1 = ctk.CTkFrame(self)
        frm2 = ctk.CTkFrame(self)
        frm3 = ctk.CTkFrame(self)
        frm1.pack(padx=5, pady=5, anchor='w', fill='x')
        frm2.pack(padx=5, pady=5, fill='both', expand=True)
        frm3.pack(padx=5, pady=5, fill='x')
        
        # ------------------------ FRAME 1 ----------------------------
        self.lblCOM = ctk.CTkLabel(frm1, text="Puerto COM:") 
        self.cboPort = ctk.CTkOptionMenu(frm1, values=['COM1', 'COM2'])
        self.lblSpace = ctk.CTkLabel(frm1, text="")
        self.btnConnect = ctk.CTkButton(frm1, text="Conectar")
        self.lblCOM.grid(row=0, column=0, padx=5, pady=5)
        self.cboPort.grid(row=0, column=1, padx=5, pady=5)
        self.lblSpace.grid(row=0,column=2, padx=30, pady=5)
        self.btnConnect.grid(row=0, column=3, padx=5, pady=5)
        
        # ------------------------ FRAME 2 ---------------------------
        self.txtChat = ctk.CTkTextbox(frm2, width=440, height=420, wrap='word', state='disable')
        self.txtChat.grid(row=0, column=0, columnspan=3, padx=5, pady=5)
                
        # ------------------------ FRAME 3 --------------------------
        self.lblText = ctk.CTkLabel(frm3, text="Texto:")
        self.inText = ctk.CTkEntry(frm3, width=240, state='disable')
        self.btnSend = ctk.CTkButton(frm3, text="Enviar", state='disable')
        self.lblText.grid(row=0, column=0, padx=5, pady=5)
        self.inText.grid(row=0, column=1, padx=5, pady=5)
        self.btnSend.grid(row=0, column=2, padx=5, pady=5)
            
        # ------------- Control del boton "X" de la ventana -----------
        self.protocol("WM_DELETE_WINDOW", self.cerrar_puertos)

        # --- ElIMINAR: LINEAS DE PRUEBA DE INSERCION DE TEXTO ---
        self.txtChat.configure(state='normal', text_color='green')
        self.txtChat.insert(0.0, f'COM1 [{datetime.now():%H:%S}]: Hola')
    
    
    def cerrar_puertos(self):
        # Se cierran los puertos COM y la ventana de tkinter
        try:
            pass
            #self.serial.close()
        except:
            pass

        self.destroy()
    
    
SerialChat().mainloop()

### Nota: threading
`threading` hace referencia a la capacidad de poder ejecutar códigos de forma concurrente, esto es de forma simultánea (pero no al mismo tiempo). De esta forma, podemos tener una operacion corriendo en paralelo a alguna otra.

Advertencia: Dependiendo del proceso que realice una función en un hilo, es posible que tenga problemas con la librería tkinter. Esto, sobretodo, si es que la función modifica los widgets del GUI.

In [23]:
import threading
import time

def func1(delay):
    for _ in range(25):
        print("*", end='')
        time.sleep(delay)

def func2(delay):
    for _ in range(25):
        print("+", end='')
        time.sleep(delay)      
        
# El keyword daemon=True permite eliminar el thread si es que el programa principal se cierra.        
th1 = threading.Thread(target=func1, args=(0.25,), daemon=True)
th1.start()

th2 = threading.Thread(target=func2, args=(0.25,), daemon=True)
th2.start()

*+